<a href="https://colab.research.google.com/github/el-eshaano/autograd/blob/main/autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import math
import torch

In [32]:
# Constants
exp = Scalar(math.exp(1))

In [80]:
def log(x: Scalar) -> Scalar:
    x = x if isinstance(x, Scalar) else Scalar(x)
    out = Scalar(math.log(x.data), (x, ), op="<Log>")

    def backward():
        x.grad += (1 / (x.data)) * out.grad
    out._backward = backward

    return out

def exp(x: Scalar) -> Scalar:
    x = x if isinstance(x, Scalar) else Scalar(x)
    out = Scalar(math.exp(x.data), (x, ), op="<Exp>")

    def backward():
        x.grad += out.data * out.grad
    out._backward = backward

    return out

In [92]:
class Scalar:

    def __init__(self, data, prev_scalars=(), op=''):
        self.data = data
        self._prev_scalars = set(prev_scalars)

        self._op = op
        self.grad = 0
        self._backward = lambda: None

    def backward(self):

        def build_topo_order(scalar, topo_order):
            for prev_scalar in scalar._prev_scalars:
                if prev_scalar not in topo_order:
                    build_topo_order(prev_scalar, topo_order)
            topo_order.append(scalar)

        topo_order = []
        build_topo_order(self, topo_order)

        self.grad = 1
        for scalar in reversed(topo_order):
            scalar._backward()

    def __repr__(self):
        return f'Scalar{{data: {self.data:.4f}, op: {self._op}}}'

    def __str__(self):
        return f"scalar{{{self.data:.4f}}}"

    def __add__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data + other.data, (self, other), '<Add>')

        def backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data * other.data, (self, other), '<Mul>')

        def backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = backward

        return out

    def __pow__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data ** other.data, (self,), f'<Pow{other.data}>')

        def backward():
            self.grad += out.grad * other * self.data**(other.data-1)
            other.grad += out.grad * log(self.data) * out.data

        out._backward = backward

        return out

    def __neg__(self):
        return self * -1

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * other**-1

    def __rtruediv__(self, other):
        return other * self**-1


In [93]:
a = Scalar(2)
b = Scalar(3)

x = 2*a + 3*b
y = 5*(a**2) + 3*(b**3)

z = 2*x + 3*y
print(z)

z.backward()

print(a.grad, b.grad)

scalar{329.0000}
scalar{64.0000} scalar{249.0000}


In [94]:
myA = Scalar(8)
myB = Scalar(2)
myOut = log(myA) / log(myB)
myOut.backward()



torchA = torch.Tensor([8]).requires_grad_()
torchB = torch.Tensor([2]).requires_grad_()
torchOut = torch.log(torchA) / torch.log(torchB)
torchOut.backward()


print(myOut)
print(torchOut)
print('---')
print(myA.grad, myB.grad)
print(torchA.grad, torchB.grad)

scalar{3.0000}
tensor([3.], grad_fn=<DivBackward0>)
---
0.18033688011112042 scalar{-2.1640}
tensor([0.1803]) tensor([-2.1640])


In [97]:
ma = Scalar(3)
mb = Scalar(2)
mo = ma ** mb
mo.backward()

ta = torch.Tensor([3]).requires_grad_()
tb = torch.Tensor([2]).requires_grad_()
to = ta ** tb
to.backward()

print(mo)
print(to)
print('----')
print(ma.grad, mb.grad)
print(ta.grad, tb.grad)

scalar{9.0000}
tensor([9.], grad_fn=<PowBackward1>)
----
scalar{6.0000} scalar{9.8875}
tensor([6.]) tensor([9.8875])
